In [3]:
import hashlib
import multiprocessing
import os
import pickle
import sys
from multiprocessing import Pool
import numpy as np
import netCDF4
from netCDF4 import Dataset
from loguru import logger
import xarray as xr
import xesmf as xe
from area import area
import pandas as pd
import f90nml
import glob
import sys
import datetime
import requests
import math
import re
import matplotlib.pyplot as plt

print(sys.version)
print('loading ok')


#################################################################################################################
#################################################################################################################

st='2019-10-12T00:00'
et='2019-10-28T12:00'


startT=pd.to_datetime(st)
endT=pd.to_datetime(et)

all_hour=int((endT-startT)/pd.Timedelta(1, 'h'))
run_hour=24*4#all_hour   #同化、重启动
run_hour=all_hour
interp_hour=run_hour #48    #同化、重启动

#修改#run_hour=all_hour    #不间断
interp_hour=run_hour

domain=2

chem_domain=[1,2,3,4,5,6]   #只要第二个domain跑chem
chem_domain=chem_domain[:domain]

history_interval_1=1*60
history_interval_2=1*60
history_interval_3=1*60

restart_interval=60*24*3

wrfreal=True
wrfda=False
restart=False
WPS= False  #do not open

WPS_epoch=True
#WPS_epoch=False
runwrf=True
genbe=False

chem=True
sgfdda=False

wrfchemi=True
wrfbio=False
wrffire=False

del_var=True


creat_wrfchemi_by_user=True

use_wrf_chem_input=False  #加入历史cheminput
use_wrf_chem_input_start=False #开始运行的时候就加入历史cheminput


cv_options=5
cv_options2=5

prepbufr=False
littler=False

if littler:
    obs_option=1
    add_obs_local=False
    add_obs_down_local=True
    down_obs=False

pswd='2000114Kk'
values = {'email' : 'xuejk2018@lzu.edu.cn', 'passwd' : pswd, 'action' : 'login'}
login_url = 'https://rda.ucar.edu/cgi-bin/login'

'''
ret = requests.post(login_url, data=values)
if ret.status_code != 200:
    print('Bad Authentication')
    print(ret.text)
    exit(1)
'''

########## PATH ######################################################################################

path_namelist='/home/xuejingkai/d1/WRF2/LQH/auto'+'/'#'/home/xuejingkai/d1/WRF/work/wrfchemcode/auto/'+'/'
#path_wrf='/exports/d1/xuejingkai/WRF2/test/runwrf/'+'/'



path_base='/exports/d4/xuejingkai/WRF/work/TW'+'/'
path_work=path_base+'/case'+startT.strftime('%Y%m%d%H')+'/'   #!!!!!!!!!!!!!!!
path_wrf=path_work+'runwrf/'

#path_wps='/exports/d1/xuejingkai/WRF2/test/wps-3.9/'+'/'    #'/home/xuejingkai/d1/WRF/work/WPS-4.3/'
path_wps=path_work+'/WPS/'+'/'    #'/home/xuejingkai/d1/WRF/work/WPS-4.3/'




###### WRFDA ###########
path_wrfda1=path_wrf+'/wrfda/'
path_wrfda2=path_wrf+'/wrfda2/'
path_wrfda3=path_wrf+'/wrfda3/'

path_obs='/home/xuejingkai/data/WRF/DATA/obs/'
path_obs_local='/home/xuejk2018/jupyter/atmos/out2/'    #自己的观测数据
path_obs_down_local='/home/xuejingkai/data/WRF/DATA/obs/'   #'/WRF-Chem/model/download_data/OBSData/' #下载的观测数据
###### WRFDA ###########

###### path of code ###########
#path_WRF='/home/xuejingkai/d1/WRF/CHEM3.9/WRFV3.9/WRFV3/' #''
path_WRF='/exports/d1/xuejingkai/WRF/CHEM3.9/WRFV3.9/WRFV3_2023'+'/'
#path_WRF='/exports/d1/xuejingkai/WRF/CHEM_wo_kpp'+'/'
#path_WRF='/exports/d1/xuejingkai/WRF/CHEM3.9/WRFV3/'

path_WRFDA='/home/xuejingkai/data/WRF/WRFDA2/'   
path_WPSDATA='/WRF-Chem/model/download_data/fnlData/'+'/'#+'fnl-'+startT.strftime('%YM%m')+'/'

path_WPS='/home/xuejingkai/d1/WRF/WPS3.9/WPS/'+'/'
path_WPS='/home/xuejingkai/data/WRF/WPS-4.3/'

###### create path of code ###########
os.system('mkdir '+path_work)
os.system('mkdir '+path_wrf)
os.system('mkdir '+path_wps)
os.chdir(path_wps)
os.system('cp -r '+path_WPS+'/* .')
os.system('rm *log*')

path_megan=path_wrf+'megan/'


path_fire=path_wrf+'fire/'

    #global path_meic,path_meic_work
path_meic='/exports/d4/xuejingkai/atmos/jupyter2/other/test/'+'meic/'
#path_meic_work= "/home/xuejingkai/data/WRF/ATMOS/meic/work3/"
path_meic_work= path_meic+"/create_chemi/"

#################################################################################################################
#################################################################################################################


3.9.9 | packaged by conda-forge | (main, Dec 20 2021, 02:40:17) 
[GCC 9.4.0]
loading ok


mkdir: cannot create directory '/exports/d4/xuejingkai/WRF/work/TW//case2019101200/': File exists
mkdir: cannot create directory '/exports/d4/xuejingkai/WRF/work/TW//case2019101200/runwrf/': File exists
mkdir: cannot create directory '/exports/d4/xuejingkai/WRF/work/TW//case2019101200//WPS//': File exists
cp: cannot stat '/home/xuejingkai/data/WRF/WPS-4.3//*': No such file or directory
rm: cannot remove 'geo.log': Permission denied
rm: cannot remove 'geogrid.log.0000': Permission denied
rm: cannot remove 'geogrid.log.0001': Permission denied
rm: cannot remove 'geogrid.log.0002': Permission denied
rm: cannot remove 'geogrid.log.0003': Permission denied
rm: cannot remove 'geogrid.log.0004': Permission denied
rm: cannot remove 'geogrid.log.0005': Permission denied
rm: cannot remove 'geogrid.log.0006': Permission denied
rm: cannot remove 'geogrid.log.0007': Permission denied
rm: cannot remove 'geogrid.log.0008': Permission denied
rm: cannot remove 'geogrid.log.0009': Permission denied
rm: 

In [4]:
!mkdir meic

mkdir: cannot create directory 'meic': Permission denied


In [9]:
################### 清单 #############################

# 排放源高度分布
emission_height_distribution = {"agriculture":   [0.7, 0.3, 0.0, 0.0, 0.0, 0.0],
                                "industry":      [0.3, 0.4, 0.2, 0.1, 0.0, 0.0],
                                "power":         [0.0, 0.1, 0.2, 0.4, 0.2, 0.1],
                                "residential":   [0.7, 0.3, 0.0, 0.0, 0.0, 0.0],
                                "transportation":[0.7, 0.3, 0.0, 0.0, 0.0, 0.0]}
# 排放源时间分布,BJT 0-23
emission_time_distribution = {"agriculture":   [0.408, 0.408, 0.432, 0.432, 0.432, 0.432, 0.72, 0.72, 0.72, 1.68, 1.68, 1.68, 2.184, 2.184, 2.184, 1.56, 1.56, 1.56, 0.6, 0.6, 0.6, 0.408, 0.408, 0.408],
                              "industry":      [0.7488, 0.7488, 0.7488, 0.7776, 0.8232, 0.888, 0.9552, 1.0296, 1.0896, 1.164, 1.2192, 1.272, 1.3032, 1.2192, 1.2384, 1.2456, 1.1592, 1.0824, 1.008, 0.9432, 0.8976, 0.8496, 0.8112, 0.7776],
                              "power":         [0.8784, 0.7848, 0.7176, 0.7176, 0.7032, 0.7416, 0.792, 0.924, 1.08, 1.1952, 1.2192, 1.2144, 1.2216, 1.1712, 1.1424, 1.1376, 1.1328, 1.0896, 1.0728, 1.0344, 1.0248, 1.0248, 1.0152, 0.9672],
                              "residential":   [0.4224,0.3768, 0.3576, 0.36, 0.3576, 0.3696, 0.4992, 1.1928, 1.5312, 1.572,1.56, 1.3584, 1.1544,1.0728,1.0608, 1.0032, 0.9792, 0.996, 1.1184, 1.4208, 1.5048, 1.3848, 1.3584, 0.9888],
                              "transportation":[0.4836, 0.4344, 0.384 , 0.36 , 0.3552, 0.3732, 0.4716, 0.6096,0.8052, 0.9888, 1.2156, 1.392 , 1.5156, 1.6164, 1.638 , 1.638 ,1.6296, 1.59, 1.4976, 1.3752, 1.1808, 0.9876, 0.8196, 0.6384]}

#transportation 普通汽车和工业用车
# [0.396 , 0.3144, 0.2424, 0.204 , 0.1944, 0.2112, 0.3168, 0.444 ,0.6384, 0.876 , 1.1592, 1.4088, 1.5912, 1.7688, 1.8072, 1.824 ,1.8264, 1.8048, 1.68  , 1.5408, 1.2888, 1.0416, 0.8256, 0.5976
#0.5712, 0.5544, 0.5256, 0.516 , 0.516 , 0.5352, 0.6264, 0.7752,0.972 , 1.1016, 1.272 , 1.3752, 1.44  , 1.464 , 1.4688, 1.452 ,1.4328, 1.3752, 1.3152, 1.2096, 1.0728, 0.9336, 0.8136, 0.6792
# 排放的周内变化。周日为0.
week_emiss_factor = {
    "agriculture":      [1.00,1.00,1.00,1.00,1.00,1.00,1.00],
    "industry":         [1.00,1.00,1.00,1.00,1.00,1.00,1.00],
    "power":            [1.00,1.00,1.00,1.00,1.00,1.00,1.00],
    "residential":      [1.00,1.00,1.00,1.00,1.00,1.00,1.00],
    "transportation":   [1.00,1.00,1.00,1.00,1.00,1.00,1.00]
}

# 无机气体的摩尔质量
inorganic_gas_mole_weight = {'CO':28, 'CO2':44, 'NH3':17, 'NOx':46, 'SO2':64}    #NOx中由10%是NO2,90%是NO,所以平均摩尔质量是46*0.1+30*0.9=31.6


'''
# 计算指定中心纬度的经纬度网格的面积。
def ll_area(lon,lat,res=0.25):
    return_area = np.zeros_like(lat)
    isize,jsize = return_area.shape
    for i in range(isize):
        for j in range(jsize):
            obj =  {'type':'Polygon','coordinates':[[[lon[i,j]-res/2.0,lat[i,j]-res/2.0],[lon[i,j]-res/2.0,lat[i,j]+res/2.0],[lon[i,j]+res/2.0,lat[i,j]+res/2.0],[lon[i,j]+res/2.0,lat[i,j]-res/2.0],[lon[i,j]-res/2.0,lat[i,j]-res/2.0]]]}
            return_area[i,j] = area(obj)/1000.0/1000.0
    return return_area
'''

# 计算指定中心纬度的经纬度网格的面积。
#已更新加速版本

def ll_area(lon,lat,res=0.25):
    return_area = np.zeros_like(lat)
    isize,jsize = return_area.shape
    gridarr=np.array([[[lon-res/2.0,lat-res/2.0],[lon-res/2.0,lat+res/2.0]
      ,[lon+res/2.0,lat+res/2.0],[lon+res/2.0,lat-res/2.0]
      ,[lon-res/2.0,lat-res/2.0]]])

    for i in range(isize):
        for j in range(jsize):
            obj =  {'type':'Polygon'
                    ,'coordinates':gridarr[...,i,j].tolist()}
            area(obj)
            return_area[i,j] = area(obj)/1000.0/1000.0
    return return_area

#插值程序,从meic网格插值到wrf网格
def meic2wrf(lon_inp,lat_inp,lon,lat,emis,interp_method = 'bilinear'):
    '''
    input:
        lon_inp: wrf的经度网格(wrfinput["XLONG"])
        lat_inp: wrf的纬度网格(wrfinput["XLAT"])
        lon: meic的经度网格,np.2darray
        lat: meic的纬度网格,np.2darray
        emis: 转化了单位后的meic排放,np.2darray
        interp_method: xesmf的插值方法
    return:
        wrf投影的排放源数据
    '''
    grid_out = {'lon': lon_inp,'lat': lat_inp}
    grid_in = {'lon': lon,'lat': lat}
    regridder = xe.Regridder(grid_in, grid_out, interp_method)#,weights="/home/tliu/data/python/emission/bilinear_200x320_179x239.nc") #reuse_weights=True
    emis_inp = regridder(emis)
    #fn = regridder.to_netcdf() #只需要一次
    return emis_inp


#插值程序,从meic网格插值到wrf网格
def meic2wrf_regridder(lon_inp,lat_inp,lon,lat,interp_method = 'bilinear'):
    '''
    input:
        lon_inp: wrf的经度网格(wrfinput["XLONG"])
        lat_inp: wrf的纬度网格(wrfinput["XLAT"])
        lon: meic的经度网格,np.2darray
        lat: meic的纬度网格,np.2darray
        emis: 转化了单位后的meic排放,np.2darray
        interp_method: xesmf的插值方法
    return:
        wrf投影的排放源数据
    '''
    grid_out = {'lon': lon_inp,'lat': lat_inp}
    grid_in = {'lon': lon,'lat': lat}
    regridder = xe.Regridder(grid_in, grid_out, interp_method)#,weights="/home/tliu/data/python/emission/bilinear_200x320_179x239.nc") #reuse_weights=True
    #emis_inp = regridder(emis)
    #fn = regridder.to_netcdf() #只需要一次
    return regridder#emis_inp

#计算一个月的有效小时数
def avg_hour(iemiss,emiss_year,emiss_month):
    avg_hour_count = 0  
    start_time = datetime.datetime(int(emiss_year),int(emiss_month),1,0)   #本月的开始时间,bjt
    while start_time.strftime("%m") == emiss_month:
        avg_hour_count += week_emiss_factor[iemiss][int(start_time.strftime("%w"))] * emission_time_distribution[iemiss][int(start_time.strftime("%H"))]
        start_time += datetime.timedelta(hours=1)
    return avg_hour_count

#物种单位转换
def convert_unit(var,value,iemiss,emiss_year="2016",emiss_month="01"):
    avg_hour_count = avg_hour(iemiss,emiss_year,emiss_month)
    #print(avg_hour_count)
    if var in ['CO', 'CO2', 'NH3', 'NOx', 'SO2', ]:  #inorganic gas: ton/(grid.month) to mole/(km2.h)
        emiss = value*1e6/(ll_area(lon,lat, 0.25)*avg_hour_count *inorganic_gas_mole_weight[var])
    elif var in ['BC', 'OC', 'PM25', 'PM10', ]:  # aerosol: ton/(grid.month) to ug/(m2.s)
        emiss = value*1e6/(ll_area(lon,lat, 0.25)*avg_hour_count*3600)
    else:  # organic gas: million_mole/(grid.month) to mole/(km2.h)
        emiss = value*1e6/(ll_area(lon,lat, 0.25)*avg_hour_count)
    return emiss

# 是否存在pickle文件，如果存在则读取
def pickle_read(pickle_file):
    flag = False
    try:
        with open(pickle_file,"rb") as f:
            return_dict = pickle.load(f)
        flag = True
        logger.success("success load "+pickle_file)
    except:
        return_dict = {}
    return flag,return_dict

#计算wrfinput文件的md5值
def md5_value(file_name):
    '''
    每一个wrfinput文件都拥有唯一的md5值
    通过md5区分不同wrfinput对应的interp_meic_emission
    '''
    with open(file_name, 'rb') as fp:
        data = fp.read()
    file_md5= hashlib.md5(data).hexdigest()
    return file_md5

#保存.pickle文件
def make_interp_meic_emission(emiss_year,emiss_month,md5value,lon_inp,lat_inp):
    pickle_dir = my_dirname+"/pickle/"
    if not os.path.exists(pickle_dir):
        os.makedirs(pickle_dir)
    
    emiss_month0=str(int(emiss_month))
    emiss_month0=emiss_month
    regridder=meic2wrf_regridder(lon_inp,lat_inp,lon,lat,interp_method = 'bilinear')
    
    
    flag,interp_meic_emission = pickle_read(pickle_dir+emiss_year+emiss_month+"_"+str(md5value)+".pickle")  #查看文件是否存在
    if not flag:
        for spec in ['SO2','NOx','NH3'##'CO2'
			,'CO','OC','BC','PM25','PM10','ALD','CH4','CSL','ETH','GLY','HC3','HC5','HC8','HCHO','ISO','KET','MACR','MGLY','MVK','NR','NVOL','OL2',"OLI","OLT","ORA1","ORA2","TOL","XYL","VOC"]:
            interp_meic_emission[spec] = {}
            for iemiss in ["agriculture","industry","power","residential","transportation"]:
                #print(iemiss)
                try:
                    #meic_emis = np.loadtxt(RADM2_DIR+"/"+emiss_year+emiss_month+"/"+emiss_year+"_"+emiss_month+"__"+iemiss+"__"+spec+".asc",skiprows = 6)[::-1,:] #行转置了 #asc
                    if spec in ['SO2','NOx','NH3','CO2','CO','OC','BC','PM25','PM10','VOC']:
                        emiss_filename = RADM2_DIR + emiss_year + "_" + emiss_month0 + "_" + iemiss + "_" + spec + ".nc"
                    else :
                        emiss_filename = RADM2_DIR + emiss_year + "_" + emiss_month0 + "_" + iemiss + "_" + "RADM2" +"_"+ spec + ".nc"
                    print(emiss_filename)
                    #print(spec)
                    ncfile = Dataset(emiss_filename, mode='r')
                    #print(ncfile.variables)
                    meic_emis = np.asarray(ncfile.variables['z'][:])
                    ncfile.close()
                    meic_emis = meic_emis.reshape(lat.shape[0], lat.shape[1])
                    meic_emis = meic_emis[::-1,:]
                    meic_emis = np.where(meic_emis > 0, meic_emis, 0)
                    meic_emis = convert_unit(var=spec,value=meic_emis,iemiss=iemiss,emiss_year=emiss_year,emiss_month=emiss_month)  #转化单位 月排放变成小时排放
                    interp_meic_emission[spec][iemiss] = {}
                    interp_meic_emission[spec][iemiss]["base"] = regridder(meic_emis)#meic2wrf(lon_inp,lat_inp,lon,lat,meic_emis,interp_method = 'bilinear') #默认 interp_method = 'bilinear' #conservative

                except:
                    print("ERROR " +spec + " " +iemiss)
                    sys.exit()
                    pass

        if not os.path.exists(pickle_dir+"/"+emiss_year+emiss_month+"_"+str(md5value)+".pickle"):
            if not os.path.exists(pickle_dir):  
                os.makedirs(pickle_dir)
            with open(pickle_dir+"/"+emiss_year+emiss_month+"_"+str(md5value)+".pickle","wb") as pickle_file:
                pickle.dump(interp_meic_emission,pickle_file)
            logger.success(pickle_dir+"/"+emiss_year+emiss_month+"_"+str(md5value)+".pickle written")
    return

#制作对应机制下排放清单
def make_wrfchemi(x):
    # x : [wrfinput_file,itime,lon_inp,lat_inp,md5value,wrfchemi_save_dir]
    wrfinput_file = x[0]
    wrf_time_utc  = x[1]
    lon_inp       = x[2]
    lat_inp       = x[3]
    md5value      = x[4]
    savedir       = x[5]
    '''
        wrfinput_file: wrfinput文件路径
        wrf_time_utc: 生成的排放源的时间
    '''
    local_t = np.floor(lon_inp / 15) + np.where(np.abs(lon_inp % 15) >= 7.5, 1, 0)
    week_emiss_local = np.zeros_like(local_t)
    emission_time_local = np.zeros_like(local_t)
    for i in range(local_t.shape[0]):
        for j in range(local_t.shape[1]):
            wrf_time_local = wrf_time_utc + datetime.timedelta(hours=int(local_t[i][j]))
            week_emiss_local[i][j] = wrf_time_local.strftime("%w")
            emission_time_local[i][j] = wrf_time_local.strftime("%H")

    wrf_time_bjt = wrf_time_utc + datetime.timedelta(hours=8)
    emiss_year = meic_year
    emiss_month = wrf_time_bjt.strftime('%m')
    domain_id = wrfinput_file.split("_")[-1][-2:]   #"01"
    logger.info("start generating " + wrf_time_utc.strftime("%Y-%m-%d_%H:00:00"))

    ncfile = savedir+"/"+wrf_time_utc.strftime("%Y/%m/%d")+"/wrfchemi_d"+domain_id+"_"+wrf_time_utc.strftime("%Y-%m-%d_%H:00:00")+""
    if os.path.exists(ncfile) and os.path.getsize(ncfile) > 56700000: #大小限制有点问题
        logger.debug(ncfile+" has been generated")
    else:
        os.system("rm -f "+ ncfile)
        with open(my_dirname+"/pickle/"+emiss_year+emiss_month+"_"+str(md5value)+".pickle","rb") as f:
            interp_meic_emission = pickle.load(f)

        #emission local time
        #读取wrf 兰博特投影网格
        for spec in interp_meic_emission.keys():
            for iemiss in ["agriculture","industry","power","residential","transportation"]:
                try:
                    #各排放源直接相加作为总排放,
                    # 系数乘在这里: 日变化，星期变化，高度层变化

                    for i in range(local_t.shape[0]):
                        for j in range(local_t.shape[1]):
                            ## 周
                            interp_meic_emission[spec][iemiss]["base"][i][j] *= week_emiss_factor[iemiss][int(week_emiss_local[i][j])]
                            ## 日变化
                            interp_meic_emission[spec][iemiss]["base"][i][j] *= emission_time_distribution[iemiss][int(emission_time_local[i][j])]
                            
                            
                            
                            
                            if spec=='NOx':
                                emission_time_distribution2=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1.072877925, 1.191637973, 1.301494371, 1.042186177, 1.033968469, 0.811988073, 0.696122852, 0.84972416, 1, 1, 1, 1, 1, 1, 1]
                                interp_meic_emission[spec][iemiss]["base"][i][j] *= emission_time_distribution2[int(emission_time_local[i][j])]
                    
                    
                    
                    
                    ##高度层变化
                    
                    interp_meic_emission[spec][iemiss]["levels"] = np.zeros((len(emission_height_distribution["power"]),lon_inp.shape[0],lon_inp.shape[1]))

                    for ilevel in range(len(emission_height_distribution["power"])):
                        interp_meic_emission[spec][iemiss]["levels"][ilevel,...] = interp_meic_emission[spec][iemiss]["base"] * emission_height_distribution[iemiss][ilevel]
                except:
                    pass

        from addict import Dict
        Meic_emission = Dict()

        PL=["agriculture","industry","power","residential","transportation"]
        LL=["E_SO2","E_NO","E_NO2","E_ALD","E_HCHO","E_ORA2","E_NH3","E_HC3","E_HC5","E_HC8","E_ETH","E_CO",
                            "E_OL2","E_OLT","E_OLI","E_TOL","E_XYL","E_KET","E_CSL","E_ISO","E_PM25I","E_PM25J","E_SO4I",
                            "E_SO4J","E_NO3I","E_NO3J","E_ORGI","E_ORGJ","E_ECI","E_ECJ","E_PM_10","E_CH3OH","E_C2H5OH","E_SO4C","E_NO3C","E_ORGC","E_ECC"]
        for spec in LL:
            for iemiss in PL:

                Meic_emission[spec][iemiss]["levels"] = np.zeros((len(emission_height_distribution["power"]), lon_inp.shape[0], lon_inp.shape[1]))

        emiss = ["agriculture", "industry", "power", "residential", "transportation"]
        for iemiss in emiss :
            Meic_emission["E_SO2"][iemiss]["levels"] = interp_meic_emission["SO2"][iemiss]["levels"]  # so2
            Meic_emission["E_NO"][iemiss]["levels"] = interp_meic_emission["NOx"][iemiss]["levels"] * 0.9  # no
            Meic_emission["E_NO2"][iemiss]["levels"] = interp_meic_emission["NOx"][iemiss]["levels"] * 0.1  # no2
            Meic_emission["E_ALD"][iemiss]["levels"] = interp_meic_emission['ALD'][iemiss]["levels"] + \
                                                       interp_meic_emission['MGLY'][iemiss]["levels"] + \
                                                       interp_meic_emission['MACR'][iemiss]["levels"] * 0.5 + \
                                                       interp_meic_emission['GLY'][iemiss]["levels"]  # ald
            Meic_emission["E_HCHO"][iemiss]["levels"] = interp_meic_emission['HCHO'][iemiss]["levels"]  # hcho
            Meic_emission["E_ORA2"][iemiss]["levels"] = interp_meic_emission['ORA2'][iemiss]["levels"]  # ora2
            Meic_emission["E_NH3"][iemiss]["levels"] = interp_meic_emission['NH3'][iemiss]["levels"]  # nh3
            Meic_emission["E_HC3"][iemiss]["levels"] = interp_meic_emission['HC3'][iemiss]["levels"]  # hc3
            Meic_emission["E_HC5"][iemiss]["levels"] = interp_meic_emission['HC5'][iemiss]["levels"]  # hc5
            Meic_emission["E_HC8"][iemiss]["levels"] = interp_meic_emission['HC8'][iemiss]["levels"]  # hc8
            Meic_emission["E_ETH"][iemiss]["levels"] = interp_meic_emission['ETH'][iemiss]["levels"]  # eth
            Meic_emission["E_CO"][iemiss]["levels"] = interp_meic_emission['CO'][iemiss]["levels"]  # co
            Meic_emission["E_OL2"][iemiss]["levels"] = interp_meic_emission['OL2'][iemiss]["levels"]  # ol2
            Meic_emission["E_OLT"][iemiss]["levels"] = interp_meic_emission['OLT'][iemiss]["levels"] + \
                                                       interp_meic_emission['MACR'][iemiss]["levels"] * 0.5 + \
                                                       interp_meic_emission['MVK'][iemiss]["levels"] * 0.5  # olt
            Meic_emission["E_OLI"][iemiss]["levels"] = interp_meic_emission['OLI'][iemiss]["levels"]  # oli
            Meic_emission["E_TOL"][iemiss]["levels"] = interp_meic_emission['TOL'][iemiss]["levels"] * 0.2950  # tol
            Meic_emission["E_XYL"][iemiss]["levels"] = interp_meic_emission['XYL'][iemiss]["levels"]  # xyl
            Meic_emission["E_KET"][iemiss]["levels"] = interp_meic_emission['KET'][iemiss]["levels"] * 0.33 + \
                                                       interp_meic_emission['MVK'][iemiss]["levels"] * 0.5  # ket
            Meic_emission["E_CSL"][iemiss]["levels"] = interp_meic_emission['CSL'][iemiss]["levels"]  # csl
            Meic_emission["E_ISO"][iemiss]["levels"] = interp_meic_emission['ISO'][iemiss]["levels"]  # iso
            Meic_emission["E_PM25I"][iemiss]["levels"] = interp_meic_emission['PM25'][iemiss]["levels"] * 0.2  # pm2.5i
            Meic_emission["E_PM25J"][iemiss]["levels"] = interp_meic_emission['PM25'][iemiss]["levels"] * 0.8  # pm2.5j

            Meic_emission["E_SO4I"][iemiss]["levels"] = interp_meic_emission['PM25'][iemiss]["levels"] * 0.036  # so4i
            Meic_emission["E_SO4J"][iemiss]["levels"] = interp_meic_emission['PM25'][iemiss]["levels"] * 0.144  # so4j
            Meic_emission["E_NO3I"][iemiss]["levels"] = interp_meic_emission['PM25'][iemiss]["levels"] * 0.016  # no3i
            Meic_emission["E_NO3J"][iemiss]["levels"] = interp_meic_emission['PM25'][iemiss]["levels"] * 0.064  # no3j

            Meic_emission["E_ORGI"][iemiss]["levels"] = interp_meic_emission['OC'][iemiss]["levels"] * 0.2  # orgi
            Meic_emission["E_ORGJ"][iemiss]["levels"] = interp_meic_emission['OC'][iemiss]["levels"] * 0.8  # orgj
            Meic_emission["E_ECI"][iemiss]["levels"] = interp_meic_emission['BC'][iemiss]["levels"] * 0.2  # eci
            Meic_emission["E_ECJ"][iemiss]["levels"] = interp_meic_emission['BC'][iemiss]["levels"] * 0.8  # ecj
            Meic_emission["E_PM_10"][iemiss]["levels"] = interp_meic_emission['PM10'][iemiss]["levels"] +interp_meic_emission['PM25'][iemiss]["levels"]  # pm10
            Meic_emission["E_CH3OH"][iemiss]["levels"] = np.zeros_like(Meic_emission["E_SO2"][iemiss]["levels"])  # CH3OH
            Meic_emission["E_C2H5OH"][iemiss]["levels"] = np.zeros_like(Meic_emission["E_SO2"][iemiss]["levels"])  # C2H5OH
            Meic_emission["E_SO4C"][iemiss]["levels"] = np.zeros_like(Meic_emission["E_SO2"][iemiss]["levels"])  # SO4C
            Meic_emission["E_NO3C"][iemiss]["levels"] = np.zeros_like(Meic_emission["E_SO2"][iemiss]["levels"])  # NO3C
            Meic_emission["E_ORGC"][iemiss]["levels"] = np.zeros_like(Meic_emission["E_SO2"][iemiss]["levels"])  # ORGC
            Meic_emission["E_ECC"][iemiss]["levels"] = np.zeros_like(Meic_emission["E_SO2"][iemiss]["levels"])  # ECC

        wrfchem_emission = {}
        for spec in Meic_emission.keys():
            Meic_emission[spec]["all"]["levels"] = np.zeros((len(emission_height_distribution["power"]), lon_inp.shape[0], lon_inp.shape[1]))
            for iemiss in ["agriculture","industry","power","residential","transportation"]:
                try:
                    Meic_emission[spec]["all"]["levels"] += Meic_emission[spec][iemiss]["levels"]
                except:
                    pass
            
            
            #--------------agriculture----------------------------
            #Meic_emission['NH3']["all"]["levels"]=Meic_emission['NH3']['agriculture']["levels"]
            #--------------agriculture----------------------------
            
            wrfchem_emission[spec]=Meic_emission[spec]["all"]["levels"]
       
        #--------------agriculture----------------------------
        #wrfchem_emission['E_NH3']=wrfchem_emission['E_NH3']*0.8    #Meic_emission['E_NH3']['agriculture']["levels"]#Meic_emission[spec]["all"]["levels"]
        #--------------agriculture----------------------------
        #wrfchem_emission['E_SO2']=wrfchem_emission['E_SO2']*2


        #生成nc文件
        ds_dict = {
            'dims': {
                'Time':None,
                'emissions_zdim_stag': wrfchem_emission["E_SO2"].shape[0],
                'south_north':wrfchem_emission["E_SO2"].shape[1],
                'west_east':wrfchem_emission["E_SO2"].shape[2],
                'DateStrLen': 19
            },
            'coords': {        
                "XLONG": {
                    'dims': ("south_north", "west_east"),
                    'attrs': {        
                        "MemoryOrder":"XY",
                        "description":"LONGITUDE, WEST IS NEGATIVE",
                        "units":"degree east",
                        "stagger":"",
                        "FieldType":104},
                    'data': lon_inp
                },
                "XLAT": {
                    'dims': ("south_north", "west_east"),
                    'attrs': {        
                        "MemoryOrder":"XY",
                        "description":"LATITUDE, SOUTH IS NEGATIVE",
                        "units":"degree north",
                        "stagger":"",
                        "FieldType":104},
                    'data': lat_inp
                }
                },
            'data_vars': { 
                "Times":{
                    'dims': ("Time"),
                    'attrs': {},
                    'data': np.array([wrf_time_utc.strftime("%Y-%m-%d_%H:%M:%S").encode("utf-8")])
                }
            }
        }

        for ivar in wrfchem_emission.keys():
            if ivar in ["E_PM25I","E_PM25J","E_SO4I","E_SO4J","E_NO3I","E_NO3J","E_ORGI","E_ORGJ","E_ECI","E_ECJ","E_PM_10","E_SO4C","E_NO3C","E_ORGC","E_ECC"]:
                ds_dict['data_vars'][ivar] = {
                        'dims': ('Time', 'emissions_zdim_stag', 'south_north', 'west_east'),
                        'attrs': {        
                            "MemoryOrder":"XYZ",
                            "description":"EMISSIONS",
                            "units":"ug m^-2 s^-1",
                            'stagger' :"Z",
                            "FieldType":104},
                        'data': wrfchem_emission[ivar][np.newaxis,...]
                }
            else:
                ds_dict['data_vars'][ivar] = {
                        'dims': ('Time', 'emissions_zdim_stag', 'south_north', 'west_east'),
                        'attrs': {        
                            "MemoryOrder":"XYZ",
                            "description":"EMISSIONS",
                            "units":"mol km^-2 hr^-1",
                            'stagger' :"Z",
                            "FieldType":104},
                        'data': wrfchem_emission[ivar][np.newaxis,...]
                }
    
        ds = xr.Dataset.from_dict(ds_dict)
        if not os.path.exists(savedir+"/"+wrf_time_utc.strftime("%Y/%m/%d")):
            try:
                os.makedirs(savedir+"/"+wrf_time_utc.strftime("%Y/%m/%d"))
            except:
                print('creat error :',savedir+"/"+wrf_time_utc.strftime("%Y/%m/%d"))
                pass
        ds.to_netcdf(savedir+"/"+wrf_time_utc.strftime("%Y/%m/%d")+"/wrfchemi_d"+domain_id+"_"+wrf_time_utc.strftime("%Y-%m-%d_%H:00:00")+"")
        # del ds_dict
        # del ds
        # gc.collect()
        logger.success("finish generating "+wrf_time_utc.strftime("%Y-%m-%d_%H:00:00"))
    return

#并行跑数据
def parallel_make_wrfchemi(start_time,end_time,n_jobs=-1):

    wrfinput_ds = xr.open_dataset(wrfinput_file)
    md5value = md5_value(wrfinput_file)
    lon_inp = wrfinput_ds['XLONG'][0, ...].values
    lat_inp = wrfinput_ds['XLAT'][0, ...].values

    timelist=[]   #需要生成排放源的时次(utc)
    while start_time <= end_time:
        timelist.append(start_time)
        start_time +=  datetime.timedelta(hours=1)

    if n_jobs == -1:
        n_jobs = multiprocessing.cpu_count()

    month_already = []
    for itime in timelist:
        itime_bjt = itime + datetime.timedelta(hours=8)  # 北京时间
        if not itime_bjt.strftime("%m") in month_already:
            make_interp_meic_emission(meic_year, itime_bjt.strftime("%m"), md5value, lon_inp, lat_inp)
            month_already.append(itime_bjt.strftime("%m"))

    all_list = []
    for itime in timelist:
        all_list.append([wrfinput_file,itime,lon_inp,lat_inp,md5value,wrfchemi_save_dir])

    
    with Pool(n_jobs) as p:
        p.map(make_wrfchemi,all_list)

def merge_wrfchemi(dom):
    attrs={'TITLE': ' OUTPUT FROM *             PROGRAM:WRF/CHEM V3.9.1 MODEL',
     'START_DATE': starttime.strftime('%Y-%m-%d_%H:%M:%S'),#'2018-12-01_00:00:00',
     'WEST-EAST_GRID_DIMENSION': wrfnamelist['domains']['e_we'][0],#240,
     'SOUTH-NORTH_GRID_DIMENSION': wrfnamelist['domains']['e_sn'][0],#180,
     'BOTTOM-TOP_GRID_DIMENSION':wrfnamelist['domains']['e_vert'][0], #45,
     'DX': wrfnamelist['domains']['dx'][0],#20000.0,
     'DY': wrfnamelist['domains']['dy'][0],#20000.0,
     'GRIDTYPE': 'C',
     'DIFF_OPT': 1,
     'KM_OPT': 4,
     'DAMP_OPT': 0,
     'DAMPCOEF': 0.01,
     'KHDIF': 0.0,
     'KVDIF': 0.0,
     'MP_PHYSICS': 2,
     'RA_LW_PHYSICS': 4,
     'RA_SW_PHYSICS': 4,
     'SF_SFCLAY_PHYSICS': 1,
     'SF_SURFACE_PHYSICS': 2,
     'BL_PBL_PHYSICS': 1,
     'CU_PHYSICS': 3,
     'SF_LAKE_PHYSICS': 0,
     'SURFACE_INPUT_SOURCE': 1,
     'SST_UPDATE': 1,
     'GRID_FDDA': 1,
     'GFDDA_INTERVAL_M': 0,
     'GFDDA_END_H': 0,
     'GRID_SFDDA': 0,
     'SGFDDA_INTERVAL_M': 0,
     'SGFDDA_END_H': 0,
     'HYPSOMETRIC_OPT': 2,
     'WEST-EAST_PATCH_START_UNSTAG': 1,
     'WEST-EAST_PATCH_END_UNSTAG': 239,
     'WEST-EAST_PATCH_START_STAG': 1,
     'WEST-EAST_PATCH_END_STAG': 240,
     'SOUTH-NORTH_PATCH_START_UNSTAG': 1,
     'SOUTH-NORTH_PATCH_END_UNSTAG': 179,
     'SOUTH-NORTH_PATCH_START_STAG': 1,
     'SOUTH-NORTH_PATCH_END_STAG': 180,
     'BOTTOM-TOP_PATCH_START_UNSTAG': 1,
     'BOTTOM-TOP_PATCH_END_UNSTAG': 44,
     'BOTTOM-TOP_PATCH_START_STAG': 1,
     'BOTTOM-TOP_PATCH_END_STAG': 45,
     'GRID_ID': 1,
     'PARENT_ID': 0,
     'I_PARENT_START': 1,
     'J_PARENT_START': 1,
     'PARENT_GRID_RATIO': 1,
     'DT': 60.0,
     'CEN_LAT': 34.999996,
     'CEN_LON': 100.0,
     'TRUELAT1': 10.0,
     'TRUELAT2': 60.0,
     'MOAD_CEN_LAT': 34.999996,
     'STAND_LON': 100.0,
     'POLE_LAT': 90.0,
     'POLE_LON': 0.0,
     'GMT': 0.0,
     'JULYR': 2018,
     'JULDAY': 335,
     'MAP_PROJ': 1,
     'MAP_PROJ_CHAR': 'Lambert Conformal',
     'MMINLU': 'MODIFIED_IGBP_MODIS_NOAH',
     'NUM_LAND_CAT': 21,
     'ISWATER': 17,
     'ISLAKE': 21,
     'ISICE': 15,
     'ISURBAN': 13,
     'ISOILWATER': 14}



    timestart=starttime.floor('d')#'2017-5-1'
    timeend=starttime.floor('d')+datetime.timedelta(hours=24)
    #dom=1
    outpath=path_meic_work+str(starttime.year)+'/'+starttime.strftime('%m')+'/'#''

    filepath=path_meic_work+starttime.strftime('%Y/%m/')#'/WRF-Chem/model/cbmz_emis/meic_2017-20km-python/'
    for i in pd.date_range(start=timestart,end=timeend,freq='d'): #periods=365,
        #print(i.strftime('%Y-%m-%d_%H:%M:%S'))
        for h in [0,12]:

            time=i+datetime.timedelta(hours=h)
            files='wrfchemi_d0'+str(int(dom))+'_'+pd.date_range(start=time,periods=12,freq='h').strftime('%Y-%m-%d_%H:%M:%S')
            #path_meic_work+i.strftime('/%Y/%m/%d/')+files
            ds=xr.open_mfdataset(path_meic_work+i.strftime('/%Y/%m/%d/')+files, concat_dim='Time',combine='nested')
            ds['Time']=pd.date_range(start=time,periods=12,freq='h')

            ds=ds.rename_dims({'emissions_zdim_stag':'emissions_zdim'})
            ds.attrs=attrs
            output_name=outpath+'wrfchemi_'+"%02d"%h+'z_d0'+str(int(dom))+'_'+time.strftime('%Y-%m-%d')

            print('Writing:',output_name)
            comp = dict(zlib=True, complevel=4)
            encoding = {var: comp for var in ds.data_vars}
            #encoding={'dis': {'zlib': True}}
            ds.to_netcdf(output_name, encoding=encoding)
            ds.close()
            file_stats = os.stat(output_name)#显示文件大小
            print(f'Success!, File Size {"%.2f"%(file_stats.st_size / (1024 * 1024))} MB')

#创建单层目录
def mkdir_single(path):
    #目录名称
    #basename:返回目录路径中的最后一个元素
    dirName = os.path.basename(path)
    # 判断路径是否存在
    isExists=os.path.exists(path)

    if not isExists:
        # 如果不存在，则创建单层目录
        os.mkdir(path) 
        print('目录创建成功：' + dirName )
        return True
    else:
        # 如果目录存在则不创建，并提示目录已存在
        print('目录已存在：' + dirName )
        return False            
def creat_wrfchemi(starttime):
    print('Creating MEIC input')
    os.chdir(path_wrf)   #创建meic生成目录
    mkdir_single('meic')

    os.system('mkdir '+path_meic_work)  #创建meic的chemi生成目录
    os.chdir(path_meic_work)

    import shutil  
    for item in glob.glob(path_meic_work+'*'):  #delete all files in path_meic_work
        print(item)
        shutil.rmtree(item, ignore_errors=True)
        

    os.chdir(path_meic)

    mkdir_single('work')


    for dom in chem_domain:
        print('wrfchemi domain: ',dom)
        my_dict = {}
        my_dict['start_time']=starttime.floor('d').strftime('%Y-%m-%dT%H')

        my_dict['end_time']=(starttime.floor('d')+datetime.timedelta(days=2)).strftime('%Y-%m-%dT%H')#endtime.strftime('%Y-%m-%dT%H')
        #my_dict['end_time']=endtime.strftime('%Y-%m-%dT%H')
        my_dict['wrfchemi_save_dir']= path_meic_work

        my_dict['my_dirname'] = path_meic+"work/" #排放所在目录名称 pickle

        #my_dict['RADM2_DIR'] = "/WRF-Chem/model/meic-original/meic_2017/RADM2/"#
        my_dict['RADM2_DIR'] = "/WRF-Chem/model/MEIC/RADM2/2017/"##"/home/xuejingkai/data/WRF/DATA/emission/MEIC/2017/07/"
        my_dict['RADM2_DIR'] = RADM2_DIR#"/WRF-Chem/model/meic-original/meic_2017/RADM2/"
        my_dict['wrfinput_file'] = path_wrf+'wrfinput_d0'+str(int(dom))  #!!!!!!!!!!!


        # 保存文件
        np.save(path_meic+'file.npy', my_dict)
        # 读取文件
        new_dict = np.load(path_meic+'file.npy', allow_pickle='TRUE')

        print(new_dict)
        #os.system('python meicrun.py')
        #os.system('cp /home/xuejingkai/d1/WRF/work/gen_emiss/meic/code/run_meic.slurm .')

        #with open('run_meic.slurm', encoding="utf-8",mode="a") as f:
        #    f.write('python meicrun.py')    

        #f.close()

        #os.system('sbatch run_meic.slurm')
            
        
        #from pyinstrument import Profiler

        #profiler = Profiler()
        #profiler.start()

        '''

        my_dirname = "/home/tliu/data/python/emission/test_nc/" #排放所在目录名称 pickle
        # 其他指定路径
        RADM2_DIR = "/home/tliu/data/python/emission/test_nc/RADM2/"  # meic RADM2文件存储路径。内部为YYYYMM的文件夹，文件夹内存放每个月都meic排放源文件
        wrfinput_file = "/home/tliu/data/python/emission/test_nc/wrfinput_d01"  # wrfinput文件位置
        wrfchemi_save_dir = "/home/tliu/data/python/emission/test_nc/"  # wrfchemi文件存放目录

        my_dirname = "/home/xuejingkai/data/WRF/ATMOS/meic/work/" #排放所在目录名称 pickle
        RADM2_DIR = "/home/xuejingkai/data/WRF/DATA/emission/MEIC/2017/06/"    #meic cb05文件存储路径。内部为YYYYMM的文件夹，文件夹内存放每个月都meic排放源文件
        wrfinput_file = "/home/xuejingkai/d1/WRF/work/wrfchem3/wrfinput_d01"   #wrfinput文件位置
        wrfchemi_save_dir = "/home/xuejingkai/data/WRF/ATMOS/meic/work/"   #wrfchemi文件存放目录
        '''
        
        ################################
        
        global lon, lat,meic_year,wrfchemi_save_dir,wrfinput_file,RADM2_DIR,my_dirname
        
        meic_dict = np.load('file.npy', allow_pickle='TRUE').tolist()
        #print(new_dict)
        my_dirname = meic_dict['my_dirname']#排放所在目录名称 pickle
        RADM2_DIR = meic_dict['RADM2_DIR']#  #meic cb05文件存储路径。内部为YYYYMM的文件夹，文件夹内存放每个月都meic排放源文件
        wrfinput_file = meic_dict['wrfinput_file']   #wrfinput文件位置
        wrfchemi_save_dir = meic_dict['wrfchemi_save_dir']  #wrfchemi文件存放目录    
        
        # meic 清单的年份
        meic_year = "2017"
        # meic经纬度,格点的位于每个网格的中心点
        lon = np.arange(70.125, 150, 0.25, dtype=np.float32)
        lat = np.arange(10.125,  60, 0.25, dtype=np.float32)
        # mix
        # lon = np.arange(40.125, 180, 0.25, dtype=np.float32)
        # lat = np.arange(-20.125, 90, 0.25, dtype=np.float32)
        lon, lat = np.meshgrid(lon, lat)
        start_time2 = pd.to_datetime(meic_dict['start_time'])#datetime(2019,6,1,0)
        end_time2 = pd.to_datetime(meic_dict['end_time'])#datetime(2019,6,10,0)
        print(wrfinput_file)
                    
        parallel_make_wrfchemi(start_time2,end_time2,n_jobs=-1)
        # 这里是你要分析的代码

        #profiler.stop()

        #profiler.print()
        
        #merge_wrfchemi(dom)        

In [16]:
RADM2_DIR="/exports/d4/xuejingkai/down/meic/RADM2/2017/"
path_wrf
path_meic


'/exports/d4/xuejingkai/atmos/jupyter2/other/test/meic/'

In [10]:
starttime=pd.to_datetime('20240401')

In [11]:
creat_wrfchemi(starttime)

Creating MEIC input
目录已存在：meic
/exports/d4/xuejingkai/atmos/jupyter2/other/test/meic//create_chemi/2024


mkdir: cannot create directory '/exports/d4/xuejingkai/atmos/jupyter2/other/test/meic//create_chemi/': File exists


目录已存在：work
wrfchemi domain:  1
{'start_time': '2024-04-01T00', 'end_time': '2024-04-03T00', 'wrfchemi_save_dir': '/exports/d4/xuejingkai/atmos/jupyter2/other/test/meic//create_chemi/', 'my_dirname': '/exports/d4/xuejingkai/atmos/jupyter2/other/test/meic/work/', 'RADM2_DIR': '/exports/d4/xuejingkai/down/meic/RADM2/2017/', 'wrfinput_file': '/exports/d4/xuejingkai/WRF/work/TW//case2019101200/runwrf/wrfinput_d01'}
/exports/d4/xuejingkai/WRF/work/TW//case2019101200/runwrf/wrfinput_d01


2024-06-20 16:12:19.123 | SUCCESS  | __main__:pickle_read:131 - success load /exports/d4/xuejingkai/atmos/jupyter2/other/test/meic/work//pickle/201704_8a08980933ea6034661e6aa723705ed7.pickle
2024-06-20 16:12:19.818 | INFO     | __main__:make_wrfchemi:223 - start generating 2024-04-01_02:00:00
2024-06-20 16:12:19.838 | INFO     | __main__:make_wrfchemi:223 - start generating 2024-04-01_19:00:00
2024-06-20 16:12:19.860 | INFO     | __main__:make_wrfchemi:223 - start generating 2024-04-01_03:00:00
2024-06-20 16:12:19.859 | INFO     | __main__:make_wrfchemi:223 - start generating 2024-04-01_01:00:00
2024-06-20 16:12:19.855 | INFO     | __main__:make_wrfchemi:223 - start generating 2024-04-01_00:00:00
2024-06-20 16:12:19.886 | INFO     | __main__:make_wrfchemi:223 - start generating 2024-04-01_11:00:00
2024-06-20 16:12:19.878 | INFO     | __main__:make_wrfchemi:223 - start generating 2024-04-01_04:00:00
2024-06-20 16:12:19.887 | INFO     | __main__:make_wrfchemi:223 - start generating 2024-

wrfchemi domain:  2
{'start_time': '2024-04-01T00', 'end_time': '2024-04-03T00', 'wrfchemi_save_dir': '/exports/d4/xuejingkai/atmos/jupyter2/other/test/meic//create_chemi/', 'my_dirname': '/exports/d4/xuejingkai/atmos/jupyter2/other/test/meic/work/', 'RADM2_DIR': '/exports/d4/xuejingkai/down/meic/RADM2/2017/', 'wrfinput_file': '/exports/d4/xuejingkai/WRF/work/TW//case2019101200/runwrf/wrfinput_d02'}
/exports/d4/xuejingkai/WRF/work/TW//case2019101200/runwrf/wrfinput_d02
/exports/d4/xuejingkai/down/meic/RADM2/2017/2017_04_agriculture_SO2.nc
ERROR SO2 agriculture



KeyboardInterrupt

